# Game plan

- i supply an openai json and extract the functions i want
- i build those functions dynamically
- i describe what the agent should do and the structured types it should hold
- i generate the final markdown
- i test that the agent works
- i save it
- i make another few
- i create a discovery system that chains them somehow

In [ ]:
from funkyprompt.core.utils.openapi import Cache
from funkyprompt.core.functions import Function

d= Cache.resolve('https://data.resmagic.io/meta-one/docs#/default/get_body_margins_meta_one_bodies_margins_get')
#d = Cache.resolve('https://data.resmagic.io/meta-one/docs#/default/get_nest_plot_for_sku_meta_one_meta_one_nest_get')
d.model_dump()

In [ ]:
F = Function.from_openapi_endpoint(d)
F(body_code='KT-2011')

In [ ]:
"""this is resolved from the default domain where the api is served"""

e = Cache.resolve_endpoint('get:/meta-one/bodies/points_of_measure')
e

In [ ]:
x = d(body_code='KT-2011')
#x = d(sku='NA-1001 LSC19 TENCVT 3ZZMD')
x

In [ ]:
from funkyprompt.core.types.markdown import MarkdownAgent

from IPython.display import Markdown

r = MarkdownAgent.build(
    'This agent is designed to interact with the Meta ONE API to perform tasks related to body measurements and related data. It can check the status of Bertha on bodies, retrieve body points of measure, style costs, and body-related margins. The agent maintains a focus on a specific body, keeping track of its name, description, measurements in inches, body code, and version. When commenting on the measurements field, add that they are referred to as points of measurements',
    'https://data.resmagic.io/meta-one/openapi.json')

Markdown(r)

# Next
- we can test parse but all we need to do is allow the agent to "activate" the fuctions it needs
- there are different animals - some objects are "just" types but they have related entity functions
- some objects are goals i.e. we are creating into types
- some objects are just function wrappers

In [ ]:
from funkyprompt.core.agents import Runner
from funkyprompt.core import AbstractModel
from IPython.display import Markdown

markdown = r# "# Body Measurement Agent\nThis agent is designed to interact with the Meta ONE API to perform tasks related to body measurements and related data. It can check the status of Bertha on bodies, retrieve body points of measure, style costs, and body-related margins. The agent maintains a focus on a specific body, keeping track of its name, description, measurements in inches, body code, and version.\n\n## Structured Response Types\n### Body Entity\n| Field Name | Type   | Description                                      |\n|------------|--------|--------------------------------------------------|\n| name       | string | The name of the body entity                      |\n| description| string | A detailed description of the body entity        |\n| measurements| array | A list of measurements in inches                 |\n| body_code  | string | The unique code identifying the body             |\n| version    | integer| The version number of the body                   |\n\n## Available Functions\n- [get:meta-one/bodies/points_of_measure](https://data.resmagic.io/meta-one/docs#/Meta%20ONE/get_body_points_of_measure_meta_one_bodies_points_of_measure_get) : Retrieve the points of measure for a body.\n- [get:meta-one/styles/costs](https://data.resmagic.io/meta-one/docs#/Meta%20ONE/get_meta_one_costs_table_using_multi_meta_one_meta_one_costs_table_get) : Fetch costs table for a style using a SKU.\n- [get:meta-one/bodies/margins](https://data.resmagic.io/meta-one/docs#/Meta%20ONE/get_body_margins_meta_one_bodies_margins_get) : Get the margins for the body.\n- [get:meta-one/body_material/margins](https://data.resmagic.io/meta-one/docs#/Meta%20ONE/get_body_material_margins_meta_one_body_material_margins_get) : Get the margins for the body material combo.\n- [get:meta-one/bertha/status](https://data.resmagic.io/meta-one/docs#/Bertha/status_meta_one_bertha_status_get) : Check Bertha's job status chart.\n- [get:meta-one/bodies/get_body_by_code](https://data.resmagic.io/meta-one/docs#/Meta%20ONE/get_body_by_code_meta_one_bodies_get_body_by_code_get) : Retrieve a body by its code.\n- [get:meta-one/bodies/status](https://data.resmagic.io/meta-one/docs#/Meta%20ONE/get_body_status_meta_one_bodies_status_get) : Check the status of the body in the queue."
model = AbstractModel.create_model_from_markdown(markdown)
agent = Runner(model)

agent

In [ ]:
Markdown(agent('Please lookup the status and measurements in the larges size for body KT-2011 and use the structured response format to response'))

In [ ]:
Markdown(agent('Please load the status and margins of by KT-3038'))

In [ ]:
Markdown(agent('Please load the body piece names (using the bodies status endpoint) for KT-2011 and also get its measurements (points of measure) for the large size and then respond in the structure respond for Body Entity'))

In [ ]:
Markdown(model.get_model_as_prompt())


In [ ]:
# prove we can parse it if we want to but we should not reall have to
# from funkyprompt.core.types.markdown import MarkdownAgent
# sample = "# Body Measurement Agent\nThis agent is designed to interact with the Meta ONE API to perform tasks related to body measurements and related data. It can check the status of Bertha on bodies, retrieve body points of measure, style costs, and body-related margins. The agent maintains a focus on a specific body, keeping track of its name, description, measurements in inches, body code, and version.\n\n## Structured Response Types\n### Body Entity\n| Field Name | Type   | Description                                      |\n|------------|--------|--------------------------------------------------|\n| name       | string | The name of the body entity                      |\n| description| string | A detailed description of the body entity        |\n| measurements| array | A list of measurements in inches                 |\n| body_code  | string | The unique code identifying the body             |\n| version    | integer| The version number of the body                   |\n\n## Available Functions\n- [get:meta-one/bodies/points_of_measure](https://data.resmagic.io/meta-one/docs#/Meta%20ONE/get_body_points_of_measure_meta_one_bodies_points_of_measure_get) : Retrieve the points of measure for a body.\n- [get:meta-one/styles/costs](https://data.resmagic.io/meta-one/docs#/Meta%20ONE/get_meta_one_costs_table_using_multi_meta_one_meta_one_costs_table_get) : Fetch costs table for a style using a SKU.\n- [get:meta-one/bodies/margins](https://data.resmagic.io/meta-one/docs#/Meta%20ONE/get_body_margins_meta_one_bodies_margins_get) : Get the margins for the body.\n- [get:meta-one/body_material/margins](https://data.resmagic.io/meta-one/docs#/Meta%20ONE/get_body_material_margins_meta_one_body_material_margins_get) : Get the margins for the body material combo.\n- [get:meta-one/bertha/status](https://data.resmagic.io/meta-one/docs#/Bertha/status_meta_one_bertha_status_get) : Check Bertha's job status chart.\n- [get:meta-one/bodies/get_body_by_code](https://data.resmagic.io/meta-one/docs#/Meta%20ONE/get_body_by_code_meta_one_bodies_get_body_by_code_get) : Retrieve a body by its code.\n- [get:meta-one/bodies/status](https://data.resmagic.io/meta-one/docs#/Meta%20ONE/get_body_status_meta_one_bodies_status_get) : Check the status of the body in the queue."

# MarkdownAgent.parse_markdown_to_agent_spec(sample).dict()

In [ ]:
# Create a session to create a thing
from funkyprompt.core import AbstractContentModel
from funkyprompt.core import AbstractEntity
from pydantic import Field
import typing
from funkyprompt.core.agents import AgentBuilder
from funkyprompt.core.agents import Runner
from funkyprompt.core import AbstractModel
from funkyprompt.core.agents import Runner
from funkyprompt.core import AbstractModel
from IPython.display import Markdown
from funkyprompt.services import entity_store
store =entity_store(AgentBuilder)

In [ ]:
#one core (e)
#Agent._register()
A = store.select_one('design.body_manager')
#this works because we support instance prompts but dont encourage it
#R = Runner(A)


In [ ]:
"""create an instance is the right thing to do but we hack support for using the AgentBuilder as it will look for an instance prompt"""
mage = A.instantiate_markdown_agent()
"""and agent save to a database can be loaded and used"""
R = Runner(mage)
Markdown(R("please get me the margins for body KT-2011"))

In [ ]:
from funkyprompt.core.agents import Runner
from funkyprompt.core import AbstractModel
from IPython.display import Markdown
agent = Runner(AgentBuilder)
agent

In [ ]:
"""first think you should always do is check the markdown prompt"""

Markdown(AgentBuilder.get_model_as_prompt())

In [ ]:
#AgentBuilder.run_search("can you find a body agent")

In [ ]:
a = agent("I need an agent to manage bodies in the design namespace - the agent should be able to get the margin and points of measurement of the body. Use a structured response of the same name as the agent with name and description and measurements field. Please save it for me ")
a

In [ ]:
from funkyprompt.core.utils.parsing import parse_fenced_code_blocks

#j = parse_fenced_code_blocks(a)
#Markdown(a)
# aa= Agent(**j)


# store.update_records(aa)

In [ ]:
A = store.select_one('design.body_manager')
#this works because we support instance prompts but dont encourage it
#R = Runner(A)
mage = A.instantiate_markdown_agent()
mage

In [ ]:
R = Runner(A)
Markdown(R("please get me the margins for body KT-2011"))

## if we can create an agent which is also and entity, it should be that loading the instance gets crud. lets think about that...


In [ ]:
bodies.get_embedding_fields()

In [ ]:
class bodies(AbstractContentModel):
    class Config:
        name:str ='body_manager'
        namespace: str = 'design'
        description: mage.get_model_description()
            
    measurements: dict
    comments: str
    version: int

b=entity_store(bodies).update_records(        bodies(name='test2', content='the elephants madness', measurements={}, version=1, comments='test'))

In [ ]:
#b=bodies(name='test', content='test', measurements={}, version=1, comments='test')
type(mage)

In [ ]:
"""if we were to just run this, the entity model would not exist
so we need to ingest some data - note also that the content should be embedded
- the markdown instance would need a register but lets assume something else has already created it ??
- we need to fully rehydrate the model including knowing it has embedding fields etc.
"""
mage.run_search("What can you tell me about cycle games actually?")
#mage.run_search("please retrieve the `test` entity")


In [ ]:
mage.run_search("please retrieve the `test` entity")

In [ ]:
entity_store(bodies).select_one('test')

In [ ]:
# semantic model for ingestion - from this we can save markdown agents while also registering data and inspecting schema 
# we can have an abstract model .export() to this format 

# models:
#     - name: bla bla
#     - description: bla bla bla
#     - types
#         name: primary
#         fields:
#             - name: 
#               description: 
#               type: 
#               relations:
#                 - 
#     - ingestion:
#         src: parquet path
#     - relations:
        

# Service discovery and AIDA

# Observations
- if another agent wrapped the call to product an output e.g. the functions, it would shroud the large json text from the output - we could also shoud internally for debuggin purposes
- id like some way to fail totally on not repeast mistakes e.g. DB errors
- 